In [ ]:
!pip install -U langchain-community langgraph langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 813.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 872.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.65
    Uninstalling langchain-core-0.3.6

In [ ]:
from typing import Annotated
import os

In [ ]:
import os
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
if "GROQ_API_KEY" not in os.environ:
    raise RuntimeError("Please set the GROQ_API_KEY environment variable before running.")

In [ ]:
from typing import Annotated
from langchain_core.tools import tool, InjectedToolCallId
from langgraph.prebuilt import InjectedState

# ← UPDATED!
from langgraph.graph import MessagesState
from langgraph.types import Command

from langgraph.graph import StateGraph, START, END

In [ ]:
from langchain_groq.chat_models import ChatGroq

groq_model = ChatGroq(
    model="llama-3.1-8b-instant",  # ← replace with your actual Groq model ID
    temperature=0.5,
    max_tokens=None,
    # api_key=os.environ["GROQ_API_KEY"]  # Not needed if env var is already set
)

In [ ]:
from typing import Annotated
from langchain_core.tools import tool, InjectedToolCallId
from langgraph.prebuilt import InjectedState

# MessagesState still comes from langgraph.graph
from langgraph.graph import MessagesState

# Command now lives in langgraph.types (not langgraph.graph)
from langgraph.types import Command

def create_handoff_tool(*, agent_name: str, description: str | None = None):
    """
    Returns a LangChain-Core Tool that, when invoked, emits a Command
    telling LangGraph to 'goto=<agent_name>' and append a 'tool' message.
    """
    tool_name = f"transfer_to_{agent_name}"
    desc = description or f"Hand off to {agent_name}."

    # 1) Define the inner Python function that actually returns a Command.
    def handoff_inner(
        state: Annotated[MessagesState, InjectedState],
        tool_call_id: Annotated[str, InjectedToolCallId],
    ) -> Command:
        tool_message = {
            "role": "tool",
            "content": f"Transferring to {agent_name} for further handling.",
            "name": tool_name,
            "tool_call_id": tool_call_id,
        }
        return Command(
            goto=agent_name,  # jump to that agent node in the graph
            update={**state, "messages": state["messages"] + [tool_message]},
            graph=Command.PARENT,  # after that agent runs, return to supervisor
        )

    # 2) Assign the function's __name__ to the desired tool name,
    #    so that the Tool registry will pick up the correct name.
    handoff_inner.__name__ = tool_name

    # 3) Decorate with @tool(description=…), _without_ passing name=…,
    #    because this version of `@tool` does not accept a name= argument.
    decorated_tool = tool(description=desc)(handoff_inner)
    return decorated_tool

# Now instantiate the two tools:
assign_to_scenario_agent = create_handoff_tool(
    agent_name="scenario_analysis_agent",
    description="Analyze the shooting scenario and return context.",
)

assign_to_specs_agent = create_handoff_tool(
    agent_name="camera_specs_agent",
    description="Given the scenario, provide DSLR settings.",
)


In [ ]:
from langgraph.prebuilt.chat_agent_executor import create_react_agent

In [ ]:
supervisor_agent = create_react_agent(
    model=groq_model,
    tools=[assign_to_scenario_agent, assign_to_specs_agent],
    prompt=(
        "You are a photography supervisor. You manage two agents:\n"
        "- scenario_analysis_agent: analyze a photography scenario (lighting, motion, environment).\n"
        "- camera_specs_agent: given a scenario, provide exact DSLR settings (lens, ISO, shutter speed, aperture).\n"
        "When the user gives input, decide which agent to call—do not answer yourself.\n"
        "Use exactly one handoff tool per turn: either transfer_to_scenario_analysis_agent "
        "or transfer_to_camera_specs_agent.\n"
        "After that worker finishes, control returns here automatically.\n"
        "Do NOT do scenario analysis or give settings yourself—only hand off."
    ),
    name="supervisor",
)


In [ ]:
scenario_analysis_agent = create_react_agent(
    model=groq_model,
    tools=[],  # this worker never calls other tools
    prompt=(
        "You are scenario_analysis_agent.\n"
        "Your job: Given a photography scenario (e.g. “shooting a hummingbird at dawn in low light”),\n"
        "analyze the environment, lighting conditions, subject motion, and dynamic range challenges.\n"
        "Do NOT supply camera settings—only describe what a photographer should watch out for."
    ),
    name="scenario_analysis_agent",
)

In [ ]:
camera_specs_agent = create_react_agent(
    model=groq_model,
    tools=[],
    prompt=(
        "You are camera_specs_agent.\n"
        "Your job: Given a photographer’s scenario or prior analysis, provide a complete DSLR configuration:\n"
        "- Lens (e.g. 24-70mm f/2.8)\n"
        "- Aperture\n"
        "- Shutter Speed\n"
        "- ISO\n"
        "- White Balance\n"
        "- Autofocus Mode (e.g. AI-Servo single-point)\n"
        "Be as precise as possible. Assume a full-frame DSLR by default."
    ),
    name="camera_specs_agent",
)

In [ ]:
supervisor_graph = (
    StateGraph(MessagesState)
    .add_node(
        supervisor_agent,
        destinations=("scenario_analysis_agent", "camera_specs_agent", END),
    )
    .add_node(scenario_analysis_agent)
    .add_node(camera_specs_agent)
    .add_edge(START, "supervisor")
    .add_edge("scenario_analysis_agent", "supervisor")
    .add_edge("camera_specs_agent", "supervisor")
    .compile()
)


In [ ]:
# ─────────────────────────────────────────────────────────
# Cell: Stream and “manually” print both dicts and AIMessage objects
# ─────────────────────────────────────────────────────────

print("\n=== Streaming a Sample Prompt (robust manual print) ===\n")

# Initialize a variable to hold the very last assistant output (if you need it later)
final_assistant_output = None

for chunk in supervisor_graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "I want to photograph a insect on leaf in medium light. "
                    "What lens and settings should I use?"
                ),
            }
        ]
    }
):
    # Each chunk is a dict whose key is the name of the agent (or 'tool') that just produced output.
    # Example keys might be ["supervisor"] for the tool message, then ["camera_specs_agent"] for the actual specs.
    present_keys = list(chunk.keys())
    print(f"Chunk produced by: {present_keys}\n")

    # Iterate over each key→payload pair (usually there's only one key per chunk)
    for agent_name, agent_payload in chunk.items():
        # We expect agent_payload to be a dict containing a "messages" list
        if not (isinstance(agent_payload, dict) and "messages" in agent_payload):
            print(f"  (Skipping payload for {agent_name}: {agent_payload})\n")
            continue

        history = agent_payload["messages"]
        if not history:
            print(f"  ({agent_name} has an empty message history)\n")
            continue

        last_msg = history[-1]

        # If it's a plain dict (e.g. {'role':..., 'content':...}), pull out keys directly:
        if isinstance(last_msg, dict):
            role = last_msg.get("role", "<unknown>")
            content = last_msg.get("content", "")
        else:
            # It's likely an AIMessage or HumanMessage (or similar LangChain BaseMessage).
            # Try to grab .role first; if not present, fall back to .type (or use "<assistant>" as default).
            role = getattr(last_msg, "role", None) or getattr(last_msg, "type", "<assistant>")
            # The message’s text is in .content
            content = getattr(last_msg, "content", str(last_msg))

        print(f"{agent_name} → {role}:\n{content}\n---\n")

        # If this is an assistant response, remember it
        if role.lower() in ("assistant", "ai", "tool"):  # adjust as needed
            final_assistant_output = content

# After the loop, `final_assistant_output` holds the last assistant (or tool) content, if you need it.
print("✅ Streaming complete.")
print("Final assistant output (if any):")
print(final_assistant_output)



=== Streaming a Sample Prompt (robust manual print) ===

Chunk produced by: ['supervisor']

supervisor → tool:
Transferring to camera_specs_agent for further handling.
---

Chunk produced by: ['camera_specs_agent']

camera_specs_agent → ai:
Given the scenario, I recommend the following DSLR configuration for photographing an insect on a leaf in medium light:

- Lens: A macro lens with a focal length of 100mm or 105mm (e.g., Canon MP-E 65mm f/2.8 1-5x or Nikon AF-S VR Micro-NIKKOR 105mm f/2.8G IF-ED) would be ideal for capturing the insect's details. However, since you didn't specify a macro lens, I'll suggest a general-purpose lens that can still provide a good working distance and a reasonable angle of view. A 50-135mm f/2.8 (e.g., Canon EF 50-135mm f/2.8L USM or Nikon AF-S NIKKOR 80-200mm f/2.8D ED VR II) would be a good alternative, but keep in mind that the working distance might be shorter, which could result in a more challenging shot.

- Aperture: Set the aperture to f/8 or f/1

Frontend


In [ ]:
pip install gradio openai-whisper torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 15.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.5 MB/

In [ ]:
import whisper
import gradio as gr

In [ ]:
# Load the Whisper model once
model = whisper.load_model("base")

def respond_to_input(text):
    response_parts = []

    for chunk in supervisor_graph.stream(
        {
            "messages": [
                {
                    "role": "user",
                    "content": text,
                }
            ]
        }
    ):
        for agent_name, agent_payload in chunk.items():
            if not (isinstance(agent_payload, dict) and "messages" in agent_payload):
                continue

            history = agent_payload["messages"]
            if not history:
                continue

            last_msg = history[-1]
            if isinstance(last_msg, dict):
                role = last_msg.get("role", "<unknown>")
                content = last_msg.get("content", "")
            else:
                role = getattr(last_msg, "role", None) or getattr(last_msg, "type", "<assistant>")
                content = getattr(last_msg, "content", str(last_msg))

            if role.lower() != "user":
                response_parts.append(content)

    final_output = "\n".join(response_parts).strip()

    with open("latest_response.txt", "w") as f:
        f.write(final_output)

    return final_output or "No response received."




def transcribe_audio(audio):
    if audio is None:
        return ""
    result = model.transcribe(audio)
    return result["text"]

custom_css = """
.custom-bg {
    background-image: url('https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fgetwallpapers.com%2Fwallpaper%2Ffull%2F1%2Fb%2F9%2F464324.jpg&f=1&nofb=1&ipt=6f4e0bac0d27c85a6d5c30b5e2fdfa202d9123ec1a66890cd365be59c7afee43');
    background-size: cover;
    background-position: center;
    min-height: 100vh;
    display: flex;
    justify-content: center;
    align-items: center;
}

.glass-card {
    background: rgba(255, 255, 255, 0.2);
    backdrop-filter: blur(10px);
    padding: 40px;
    border-radius: 20px;
    width: 90%;
    max-width: 500px;
    box-shadow: 0 8px 32px rgba(0,0,0,0.3);
}

#mytextbox {
    background-color: #cccccc !important;
    padding: 12px !important;
    border-radius: 10px !important;
}

#mytextbox textarea,
#mytextbox input {
    background: transparent !important;
    border: none !important;
    resize: none !important;
    color: #000 !important;
    font-size: 16px !important;
    line-height: 1.4 !important;
}

#mytextbox textarea::placeholder,
#mytextbox input::placeholder {
    color: #e0e0e0 !important;
}

#mytextbox label {
    display: block;
    margin-bottom: 6px;
    color: #fff !important;
    font-weight: 500;
}
"""

with gr.Blocks(css=custom_css) as demo:
    with gr.Row(elem_classes="custom-bg"):
        with gr.Column(elem_classes="glass-card"):
            gr.Markdown("## <div style='text-align: center;'>PHOTOGRAPHY ASSISTANT </div>")

            input_text = gr.Textbox(label="Type something…", elem_id="mytextbox")
            submit_btn = gr.Button("Submit")
            gr.Markdown("## ---------------------------Or----------------------------")
            audio_input = gr.Audio(sources="microphone", type="filepath", label="Speak instead")
            transcribe_btn = gr.Button("🎙️ Transcribe")

            output_text = gr.Textbox(label="Output", elem_id="mytextbox", interactive=False)

            # FIXED: Transcribed text now goes to output_text
            transcribe_btn.click(
    fn=transcribe_audio,
    inputs=audio_input,
    outputs=input_text  # populate the input box
)

            # Textbox input still controls output normally
            submit_btn.click(fn=respond_to_input, inputs=input_text, outputs=output_text)



if __name__ == "__main__":
    demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://109444bed2bc9b899c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
